# Import Libraries

In [1]:
import os
import xarray as xr
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
os.chdir('/Users/sabbirdelowar/projects/climate-hazard')

# Define Paths

In [3]:
data_dir = "data/interim/heat"
aoi_shp = "data/aoi/aoi-multipoly.shp"
output_csv = "results/individual-hazards/heat-hazard.csv"

years = ["2020", "2021", "2022", "2023", "2024"]
threshold_C = 35

# Load districts shapefile

In [4]:
districts = gpd.read_file(aoi_shp)
districts = districts.to_crs("EPSG:4326")
districts = districts[['name', 'geometry']].copy()
districts['heatwave_days'] = 0

In [15]:
heat_nc = xr.open_dataset("data/interim/heat/era5-daily-max-2020.nc")
print (heat_nc)
temp_K = heat_nc['t2m']
print(temp_K.min().item(), temp_K.max().item())

<xarray.Dataset>
Dimensions:     (valid_time: 31, latitude: 61, longitude: 47)
Coordinates:
    number      int64 ...
  * latitude    (latitude) float64 26.59 26.49 26.39 26.29 ... 20.79 20.69 20.59
  * longitude   (longitude) float64 88.0 88.1 88.2 88.3 ... 92.3 92.4 92.5 92.6
  * valid_time  (valid_time) datetime64[ns] 2024-01-01 2024-01-02 ... 2024-01-31
Data variables:
    t2m         (valid_time, latitude, longitude) float32 ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-07-05T21:53 GRIB to CDM+CF via cfgrib-0.9.1...
284.8681640625 301.249755859375


# Collect district daily means

In [8]:
import pandas as pd

all_records = []

for year in years:
    nc_file = os.path.join(data_dir, f"era5-daily-max-{year}.nc")
    print(f"✅ Reading {nc_file}")

    ds = xr.open_dataset(nc_file)

    # Get temperature variable
    if '2m_air_temperature_maximum' in ds.data_vars:
        temp_K = ds['2m_air_temperature_maximum']
    elif 't2m' in ds.data_vars:
        temp_K = ds['t2m']
    else:
        raise ValueError("No valid temperature variable found!")

    temp_C = temp_K - 273.15

    for single_day in tqdm(temp_C['valid_time'].values, desc=f"Year {year}"):
        day_slice = temp_C.sel(valid_time=single_day)
        day_tif = os.path.join(data_dir, "temp_day.tif")
        day_slice.rio.to_raster(day_tif)

        with rasterio.open(day_tif) as src:
            for idx, row in districts.iterrows():
                geom = [row.geometry]
                try:
                    out_image, _ = rasterio.mask.mask(src, geom, crop=True)
                    data = out_image[0]
                    data = data[data > -100]

                    if data.size > 0:
                        mean_val = data.mean()
                        all_records.append({
                            'date': pd.to_datetime(single_day),
                            'district': row['name'],
                            'mean_temp': mean_val
                        })
                except ValueError:
                    continue

        os.remove(day_tif)

✅ Reading data/interim/heat/era5-daily-max-2020.nc


Year 2020: 100%|█| 366/366 [0


✅ Reading data/interim/heat/era5-daily-max-2021.nc


Year 2021: 100%|█| 365/365 [07:36<


✅ Reading data/interim/heat/era5-daily-max-2022.nc


Year 2022: 100%|█| 365/365 [07:34<


✅ Reading data/interim/heat/era5-daily-max-2023.nc


Year 2023: 100%|█| 365/365 [07:24<


✅ Reading data/interim/heat/era5-daily-max-2024.nc


Year 2024: 100%|█| 366/366 [07:31<


# Compute 75th percentile threshold

In [10]:
df = pd.DataFrame(all_records)

HEATWAVE_THRESHOLD_C = np.percentile(df['mean_temp'], 75)
print(f"✅ Data-derived heatwave threshold (75th percentile): {HEATWAVE_THRESHOLD_C:.2f} °C")

✅ Data-derived heatwave threshold (75th percentile): 12.46 °C


# Count heatwave days per district

In [11]:
counts = df[df['mean_temp'] >= HEATWAVE_THRESHOLD_C].groupby('district').size()
counts = counts.reindex(districts['name']).fillna(0).astype(int)

districts['heatwave_days'] = counts.values

# Rescale to 0–1 hazard

In [12]:
min_days = districts['heatwave_days'].min()
max_days = districts['heatwave_days'].max()

districts['heat_hazard'] = (
    (districts['heatwave_days'] - min_days) / (max_days - min_days)
).clip(0, 1)

districts[['name', 'heatwave_days', 'heat_hazard']].rename(
    columns={'name':'district'}
).to_csv(output_csv, index=False)

# Process each year

In [ ]:
for year in years:
    nc_file = os.path.join(data_dir, f"era5-daily-max-{year}.nc")

    # Load NetCDF
    ds = xr.open_dataset(nc_file)

    # Get variable
    if '2m_air_temperature_maximum' in ds.data_vars:
        temp_K = ds['2m_air_temperature_maximum']
    elif 't2m' in ds.data_vars:
        temp_K = ds['t2m']
    else:
        raise ValueError("No valid temperature variable found!")

    # Convert to Celsius
    temp_C = temp_K - 273.15

    # Loop over days
    time_dim = temp_C['valid_time']
    
    for single_day in tqdm(time_dim.values, desc=f"Year {year}"):
        day_slice = temp_C.sel(valid_time=single_day)
        day_tif = os.path.join(data_dir, "temp_day.tif")
        day_slice.rio.to_raster(day_tif)

        # Open with Rasterio for masking
        with rasterio.open(day_tif) as src:
            for idx, row in districts.iterrows():
                geom = [row.geometry]
                try:
                    out_image, _ = rasterio.mask.mask(src, geom, crop=True)
                    data = out_image[0]
                    data = data[data > -100]  # exclude nodata

                    if data.size > 0:
                        mean_temp = data.mean()
                        if mean_temp >= threshold_C:
                            districts.loc[idx, 'heatwave_days'] += 1

                except ValueError:
                    continue  # skip geometries outside raster

        os.remove(day_tif)